## This notebook check the M2 state transitions and force commands

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

#import os
#print(os.environ["OSPL_URI"])
#print(os.environ["LSST_DDS_ALIGNER"]) #this should give an keyError. If not, something wrong with the DDS

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
#await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
m2 = salobj.Remote(script.domain, "MTM2")
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await m2.start_task

In [ ]:
#As long as you get something for the payload its OK. we dont' care about h.heartbeat
await m2.evt_heartbeat.next(flush=True, timeout=5)

### test state transition. If m2 is already enabled, disable then enable it.

In [ ]:
state = await m2.evt_summaryState.aget(timeout=5)
print('staring with: m2 state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))
if state.summaryState == 2:
    await salobj.set_summary_state(m2, salobj.State.DISABLED) #disable m2
    await asyncio.sleep(3)
    state = await m2.evt_summaryState.aget(timeout=5)
    print('m2 state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))

await salobj.set_summary_state(m2, salobj.State.ENABLED) #enable m2
state = await m2.evt_summaryState.aget(timeout=5)
print('m2 state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))

In [ ]:
#may need to wait a few seconds before event shows up in EFD
await asyncio.sleep(5.)
from lsst_efd_client import EfdClient

client = EfdClient('ncsa_teststand_efd')
csc_index = 1
end = Time(datetime.now())
start = end - timedelta(hours=1)
dfe = await client.select_time_series('lsst.sal.MTM2.logevent_summaryState', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
#to check messages in Kafka, go to https://lsst-kafka-0-nts-efd.ncsa.illinois.edu/
#we cannot get time series data from DDS. We have to query the EFD

In [ ]:
dfe

### check the forces

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

In [ ]:
fa = [0]*72
ft = [0]*6
await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)

In [ ]:
axialForces = await m2.tel_axialForce.next(flush=True, timeout=2)
tangentForces = await m2.tel_tangentForce.next(flush=True, timeout=2)

In [ ]:
def plotForces(axialForces, tangentForces):
    fig, ax = plt.subplots(2,1, figsize=(15,8))
    ax[0].plot(axialForces.measured, label='measured');
    ax[0].plot(axialForces.applied, label='applied');
    ax[0].plot(axialForces.hardpointCorrection,'.', label='FB');
    ax[0].plot(axialForces.lutGravity, label='LUT G');
    ax[0].legend()
    ax[1].plot(tangentForces.measured, label='measured');
    ax[1].plot(tangentForces.applied, label='applied');
    ax[1].plot(tangentForces.hardpointCorrection, 'o', label='FB');
    ax[1].plot(tangentForces.lutGravity, label='LUT G');
    ax[1].legend()

    aa = np.loadtxt('%s/notebooks/M2_FEA/data/M2_1um_72_force.txt'%(os.environ["HOME"]))
    # to have +x going to right, and +y going up, we need to transpose and reverse x and y
    xact = -aa[:,2]
    yact = -aa[:,1]

    fig2, ax=plt.subplots( 1,2, figsize = [10,4])
    aa = np.array(axialForces.measured)
    img = ax[0].scatter(xact, yact, c=aa, s=abs(aa)*2)
    #plt.jet()
    ax[0].axis('equal')
    ax[0].set_title('measured forces')
    fig.colorbar(img, ax=ax[0])

    aa = np.array(axialForces.applied)
    img = ax[1].scatter(xact, yact, c=aa, s=abs(aa)*2)
    #plt.jet()
    ax[1].axis('equal')
    ax[1].set_title('applied forces')
    fig.colorbar(img, ax=ax[1])
    
plotForces(axialForces, tangentForces)

In [ ]:
fa = [2]*72
ft = [-2]*6
await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)

In [ ]:
axialForces = await m2.tel_axialForce.next(flush=True, timeout=2)
tangentForces = await m2.tel_tangentForce.next(flush=True, timeout=2)
plotForces(axialForces, tangentForces)

In [ ]:
zAngle = await m2.tel_zenithAngle.aget(timeout=5)
print('zenith angle = ', zAngle.measured)

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m2, salobj.State.ENABLED) #enable m2